In [1]:
%pip install datasets
%pip install jsonlines

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.Requirement already satisfied: jsonlines in c:\users\user\projectai\.env\lib\site-packages (4.0.0)




[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


Import del dataset

In [2]:
import json
import pandas as pd
import os

data_path = '../data/'
dataset_path = os.path.join(os.getcwd(), data_path + 'annotations.jsonl')

data = []

with open(dataset_path, 'r',encoding='utf-8') as f:
    for line in f:
        data.append(json.loads(line))


Prendere colonne e metterle in variabili separate

In [3]:
df = pd.DataFrame(data)
id_tmp = df['id'].astype(str)
text_tmp = df['text'].astype(str)
label_tmp = df['label'].astype(str)
df.drop(['Comments'], axis=1)

,id,text,label
0,1,La giornata spuntò serena e limpida per gli sp...,"[[44, 49, PER], [87, 91, GPE], [132, 142, GPE]..."
1,2,I .\nLa ditta portava il nome del padre Giovan...,"[[7, 12, ORG], [33, 58, PER], [83, 107, PER], ..."
2,3,I .\nLa prima volta che Cesare Lascaris entrò ...,"[[23, 38, PER], [48, 70, FAC], [59, 70, PER], ..."
3,4,"I .\nDinanzi all ' osteria di Muzio Scevola , ...","[[18, 42, FAC], [48, 58, LOC], [139, 153, GPE]..."
4,5,"Anna così racconta : Per lungo tempo , l ' amo...","[[0, 4, PER], [64, 80, PER], [151, 179, PER], ..."
...,...,...,...
95,96,"I .\nEllade , giovinezza del mondo .\nNel temp...","[[4, 10, GPE], [28, 33, LOC], [72, 78, PER], [..."
96,97,CAPITOLO PRIMO .\nNel quale Phileas Fogg e Gam...,"[[27, 39, PER], [42, 52, PER], [85, 88, PER], ..."
97,98,"Clotilde entrò un poco sbadatamente , cantando...","[[0, 8, PER], [53, 83, FAC], [75, 83, FAC], [1..."
98,99,"I .\nSenza fiori nascosti nella sottoveste , m...","[[125, 138, PER], [200, 207, PER], [366, 372, ..."


Prendere da text parola che inizia e finisce con indice di label

In [4]:
print(df.columns)

Index(['id', 'text', 'label', 'Comments'], dtype='object')


In [12]:
import pandas as pd
import re
import jsonlines
import nltk
import os

nltk.download('punkt')
from nltk.tokenize import sent_tokenize

pattern = r'\[(\d+), (\d+), \'(\w+)\'\]'
lst = []

output_path = os.path.join(os.getcwd(), data_path, 'datasetT5.jsonl')

with jsonlines.open(output_path, 'w') as writer:

    for index, sub in enumerate(label_tmp):
        lst = re.findall(pattern, str(sub))
        text = text_tmp[index]

        # Dividi il testo in frasi
        sentences = sent_tokenize(text)

        for sentence in sentences:
            sentence_targets = []
            for item in lst:
                start_index = int(item[0])
                end_index = int(item[1])
                entity_type = item[2]
                entity_name = text[start_index:end_index + 1]

                # Verifica se l'entità è all'interno della frase corrente
                if start_index >= text.index(sentence) and end_index <= text.index(sentence) + len(sentence):
                    sentence_targets.append(entity_name + " [" + entity_type + "]")

            # Se ci sono entità nella frase corrente, crea una coppia source/target
            if sentence_targets:
                # Unisci i target in una singola stringa
                target_str = " ".join(sentence_targets)
                writer.write({'source': sentence, 'target': target_str})


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\USER\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [8]:
#testo intero
import pandas as pd
import re
import jsonlines

pattern = r'\[(\d+), (\d+), \'(\w+)\'\]'
lst = []

source = []
targets = []

output_path = os.path.join(os.getcwd(), data_path + 'datasetT5.jsonl')

with jsonlines.open(output_path, 'w') as writer:

    for index, sub in enumerate(label_tmp):    
        lst = re.findall(pattern, str(sub))
        source = text_tmp[index]
        for item in lst:
            text_tmp[index] = text_tmp[index].replace('\"','"')
            start_index = int(item[0])
            end_index = int(item[1])
            if start_index > 0 and text_tmp[index][start_index - 1] == " ":  
                start_index -= 1  # Decrementa l'indice di inizio di 1
                end_index -= 1  # Decrementa l'indice di fine di 1
            if end_index < len(text_tmp[index]) - 1 and text_tmp[index][end_index + 1] == " ":  # Se il carattere successivo è uno spazio
                end_index += 1  # Incrementa l'indice di fine di 1
            text_tmp_substring = text_tmp[index][start_index:end_index + 1]  # Incrementa di 1 l'indice di fine
            targets.append(text_tmp_substring + " [" + item[2] + "]")
            
        writer.write({'source': source, 'targets': targets})
        
        targets = []
